In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import math


from sklearn.tree            import DecisionTreeRegressor
from sklearn.linear_model    import LinearRegression,Ridge,Lasso,RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble        import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold,train_test_split 
from sklearn.metrics         import r2_score,mean_squared_error
from sklearn.ensemble        import RandomForestRegressor,  GradientBoostingRegressor 
from xgboost                 import XGBRegressor 
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV




C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [24]:
train_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\train_processedWithLog1pTarget.csv")

In [25]:
test_df = pd.read_csv("C:\\Users\\Dashang\\Downloads\\Github\\Stayze_Rent_Predicition\\Stayze_Rent_Predicition_Hackathon\\Dashang\\data\\test_processed.csv")

In [26]:
train_df.isnull().sum()

latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
log_(price+1)                        0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [27]:
train_df_with_dummies = train_df

In [28]:
X=train_df_with_dummies.drop(['log_(price+1)'],axis=1)
y=train_df_with_dummies['log_(price+1)']

In [29]:
#y= target
#x = nums
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=24)

In [30]:
xgbmodel = XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=[0,1],
             importance_type='gain', learning_rate=[0.1,0.2], max_delta_step=0,
             max_depth=11, max_leaf_nodes=16, min_child_weight=1,
             min_samples_leaf=55, min_samples_split=110, missing=None,
             n_estimators=50, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

param_grid = dict(n_estimators=range(10,100,10),
                  max_depth=range(3,20),
                  min_samples_split=range(50,500,20),
                  min_samples_leaf=range(25,75,10),
                  max_leaf_nodes=range(8,32,2))

grid = RandomizedSearchCV(xgbmodel,param_grid,scoring='neg_mean_squared_error',n_jobs=2,cv=5,random_state=33)

grid.best_estimator_

grid.best_params_


In [31]:
xgbmodel.fit(X_train,y_train)



[10:36:04] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBoostError: Invalid Parameter format for learning_rate expect float but value='[0.1, 0.2]'

In [10]:
print('Train RMSE:',np.sqrt(mean_squared_error(y_train,np.exp(xgbmodel.predict(X_train))-1)))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test,np.exp(xgbmodel.predict(X_test))-1)))

Train RMSE: 169.3179161166229
Test RMSE: 145.69824351523886


xgbmodel = XGBRegressor(n_estimators=1000, learning_rate=0.1, early_stopping=5, max_depth=5, min_child_weight=1)

xgbmodel.fit(X_train,y_train)

y_pred = xgbmodel.predict(X_test)
print('Train RMSE:',np.sqrt(mean_squared_error(y_train, xgbmodel.predict(X_train))))
print('Test RMSE:',np.sqrt(mean_squared_error(y_test, xgbmodel.predict(X_test))))

np.sqrt(mean_squared_error(y_train,np.exp(xgbmodel.predict(X_train))-1)), np.sqrt(mean_squared_error(y_test,np.exp(xgbmodel.predict(X_test))-1))

## TEST DATA

In [11]:
test_df

,id,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,1525602,40.66751,-73.95867,2,121,1.70,1,39,0,1,0,0,0,1,0,0
1,30430185,40.75655,-73.99690,1,18,2.49,30,364,0,0,1,0,0,0,1,0
2,21354525,40.69252,-73.99121,1,87,4.29,1,108,0,1,0,0,0,0,1,0
3,35995074,40.77292,-73.90101,9,0,0.01,2,365,0,0,0,1,0,0,1,0
4,34392081,40.71863,-73.94980,1,28,14.00,2,20,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14664,27730191,40.59446,-73.94289,2,1,0.59,1,36,0,1,0,0,0,1,0,0
14665,29676417,40.74319,-73.98074,2,1,0.16,1,365,0,0,1,0,0,0,0,1
14666,1813829,40.82519,-73.94530,1,93,1.34,3,358,0,0,1,0,0,0,1,0
14667,25017403,40.72238,-73.99466,1,15,1.12,8,34,0,0,1,0,0,1,0,0


In [12]:
test_id_col = test_df['id']

In [13]:
test_df.isnull().sum()

id                                   0
latitude                             0
longitude                            0
minimum_nights                       0
number_of_reviews                    0
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
neighbourhood_group_Bronx            0
neighbourhood_group_Brooklyn         0
neighbourhood_group_Manhattan        0
neighbourhood_group_Queens           0
neighbourhood_group_Staten Island    0
room_type_Entire home/apt            0
room_type_Private room               0
room_type_Shared room                0
dtype: int64

In [14]:
test_df_with_dummies = test_df

In [15]:
test_df_with_dummies = test_df_with_dummies.drop(['id'],axis=1)

In [16]:
test_df_with_dummies.columns

Index(['latitude', 'longitude', 'minimum_nights', 'number_of_reviews',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'neighbourhood_group_Bronx',
       'neighbourhood_group_Brooklyn', 'neighbourhood_group_Manhattan',
       'neighbourhood_group_Queens', 'neighbourhood_group_Staten Island',
       'room_type_Entire home/apt', 'room_type_Private room',
       'room_type_Shared room'],
      dtype='object')

In [17]:
test_df_with_dummies['price']= xgbmodel.predict(test_df_with_dummies)

In [18]:
test_df_with_dummies['price'] = np.exp(test_df_with_dummies['price'])-1

In [19]:
submissions_xgbmodel = pd.concat([test_id_col, test_df_with_dummies['price']], axis=1) 


In [20]:
submissions_xgbmodel.to_csv('submission_xgbmodel_best_est_randomSearchCV.csv', index=False) 

In [21]:
submissions_xgbmodel

,id,price
0,1525602,137.087021
1,30430185,199.787964
2,21354525,89.474266
3,35995074,70.280037
4,34392081,71.086708
...,...,...
14664,27730191,106.658806
14665,29676417,91.464645
14666,1813829,70.825668
14667,25017403,225.810364


In [22]:
submissions_xgbmodel.isnull().sum()

id       0
price    0
dtype: int64